In [1]:
pack <- c("tidyverse", "plyr", "dplyr", "np", "FNN", "modelr", "rpart", "doParallel", "sandwich",
          "grf", "Matching", "boot", "stargazer")
invisible(suppressMessages(suppressWarnings(lapply(pack, require, character.only = TRUE))))

$\newcommand{\E}{{\rm I\kern-.3em E}}$
$\newcommand{\Var}{\mathrm{Var}}$
$\newcommand{\Cov}{\mathrm{Cov}}$
$\newcommand{\Covh}{\widehat{\Cov}}$
$\newcommand{\Varh}{\widehat{\Var}}$
$\newcommand{\betah}{\widehat{\beta}}$
$\newcommand{\Eh}{\widehat{\E}}$
$\newcommand{\YO}{Y(0)}$
$\newcommand{\YI}{Y(1)}$
$\newcommand{\indep}{\perp \!\!\! \perp}$


# Settings+ Assumptions

Suppose we have random sample (i.i.d.) of the reference population, each containing a feature vector $X_i$, a treatment indicator $D_i$ and an outcome $Y_i$. In short: $\left\{(Y_i,X_i,D_i)\right\}_{i=1}^{n}$.
Generally, we will either assume unconfoundedness (ignorability) or purely random treatment assignment in our DGPs. More formally, random treatment assignment amounts to 
<br>
&ensp;
\begin{equation}
(\YI,\YO)\enspace\indep\enspace D.\label{eq:rt}
\end{equation}
<br>
&ensp;
In situations usually analyzed within the social sciences, this assumption turns out to be too restrictive (ref.). Therefore, as we've seen throughout the course, many methods were developed under the assumption of unconfoundedness:
<br>
&ensp;
\begin{equation}
(\YI,\YO)\enspace\indep\enspace D\enspace \lvert\enspace X.\label{eq:uf}
\end{equation}
<br>
&ensp;
This is also used by [@wager2018estimation] to develop the theory of causal trees/forests.\newline
Within the simulation study we'll also try to stress the consequences for estimation when switching from (\ref{eq:rt}) to (\ref{eq:uf}). 


## Heterogeneous Treatment Effects
To develop an intuition for the importance of heterogeneity in treatment effects, we define the Average Treatment Effect (ATE) and the Conditional Average Treatment Effect (CATE) as follows.
<br>
&ensp;
\begin{align}
\delta&\equiv \E(\YI-\YO)\label{eq:ate}\\[10pt]
\delta(x)&\equiv\E(\YI-\YO\vert X=x)\label{eq:cate}\\[10pt]
\end{align}
<br>
&ensp;
These definitions are conceptually different: While the CATE is a real-valued function mapping a realization of the random variable $X$ to a real number, the ATE is a real number. Thus, by definition, the CATE allows to have *distinct* treatment effects for different realizations of $X$. By the law of iterated expectation we have the following relationship:


\begin{equation*}
\E\left(\delta(X)\right)=\delta
\end{equation*}

<br>
&ensp;
Thus, the ATE is a summary statistic of the CATE. Example for illustration + ref.  


## Structural representation

For the first part of the simulations, we can postulate the following structural equations.


\begin{align*}
X&= f_{X}(u_1),\qquad f_{X}:\mathbb{R} \longrightarrow \mathbb{R},\quad x \mapsto x, \qquad u_1 \stackrel{\text{i.i.d}}{\sim} U[0,1]\\
D&= f_{D}(X)\\
Y&= f_{Y}(X, D, \varepsilon), \qquad \varepsilon \stackrel{\text{i.i.d}}{\sim} \mathcal{N}(0,1)\\
\end{align*}
The corresponding graph looks as follows.



<img src="Causal_graphs.png" height=300 width=300 />

## First Model

For the whole first part of the simulations, I'll stick to models that are linear in parameters when it comes to generating the potential outcomes. More specifically,
<br>
&ensp;
\begin{align}
Y(0)&=\gamma_0 + \gamma_1\cdot g(X)+ \gamma_2\cdot h(\gamma_3 X) +\varepsilon \\
Y(1)&=\phi_0 + \phi_1\cdot f(X)+ \phi_2\cdot c(\phi_3 X)+\varepsilon,
\end{align}
<br>
&ensp;
where

* $\varepsilon \stackrel{\text{i.i.d}}{\sim} \mathcal{N}(0,1),\quad X \stackrel{\text{i.i.d}}{\sim} U[0,1]$
<br>
&ensp;
* $g(\cdot),h(\cdot),f(\cdot),\enspace \text{and}\enspace c(\cdot)$ are real-valued functions
<br>
&ensp;
* $\phi_j, \gamma_j \in \mathbb{R}$, $j=1,2,3$
<br>
&ensp;


In this model, ATE (\ref{eq:ate}) and CATE (\ref{eq:cate}) become
<br>
&ensp;
\begin{align*}
\delta&=\phi_0 - \gamma_0 + \phi_1 \E(f(X))-\gamma_1 \E(g(X))+\phi_2\E(c(X))-\gamma_2\E(h(X))\\[5pt]
\delta(x)&=\phi_0 - \gamma_0 + \phi_1 f(x)-\gamma_1 g(x)+\phi_2 c(x)-\gamma_2 h(x).
\end{align*}
<br>
&ensp;
The workhorse function to generate the simulations from the first part looks as follows.

In [2]:
sim <- function(n, het_linear = FALSE, random_assignment = TRUE,
                non_linearY = FALSE, non_linearD = FALSE,
                gamma0=1, gamma1=3, gamma2=0, gamma3=1,
                phi0=5, phi1=3, phi1p=5, phi2=0, phi3=5){
  x <- runif(n)
  eps <- rnorm(n)
  #
  if(random_assignment){D <- rbinom(n,size=1,prob=0.5)}
  else{
    D <- rep(0,n)
    if(non_linearD){
      c_1<- 0.1; c_2 <- 0.89
      prt <- function(x){c_1+0.01*x+c_2*sin(pi*x)}
    }
    else{prt <- function(x){x}}
    for(j in seq_along(D)){D[j] <- rbinom(1,size=1,prob=prt(x[j]))}
  }
  if(non_linearY){
    nl <- function(z,const,t=1){const+3*z+20*sin(pi*z*t)}
    y1 <- nl(z=x,const=phi0,t=phi3) + eps
    y0 <- nl(z=x,const=gamma0,t=gamma3) + eps
  }
  else{
    if(het_linear){y1 <- phi0 + phi1p*x + eps}
    else{y1 <- phi0 + phi1*x + eps}
    y0 <- gamma0 + gamma1*x + eps
  }
  res <- tibble(Y0=y0,Y1=y1,X=x,D=D,Y_obs=rep(0,n),IntXD=x*D)
  res$Y_obs[res$D==1] <- res$Y1[res$D==1]
  res$Y_obs[res$D==0] <- res$Y0[res$D==0]
  return(res)
}

The function allows you to tweak a variety of parameters. 
* $n$ is the number of observations
<br>
&nbsp;
* `het_linear` is boolean and leads to different slopes of $Y(0)$ and $Y(1)$ in $X$
<br>
&nbsp;
* `random_assignment`is boolean and allows you to switch between random treatment assignment ($\ref{eq:rt}$) and unconfoundedness ($\ref{eq:uf}$).<br>
<br>
&nbsp;
Under random treatment assignment $\text{Pr}(D\vert X=x)=\frac{1}{2}$, whereas under unconfoundedness I hoose a linearly increasing function: $\text{Pr}(D\vert X=x)=x$
<br>
&nbsp;
* `non_linearY` is boolean and implements non-linear $Y(1)$ and $Y(0)$ by specifying 
<br>
&nbsp;
\begin{align*}
h&:\mathbb{R} \longrightarrow \mathbb{R},\quad x \mapsto \sin(\pi\gamma_3 x)\\
c&:\mathbb{R} \longrightarrow \mathbb{R},\quad x \mapsto \sin(\pi\phi_3 x)
\end{align*}
<br>
&nbsp;
* `non_linearD` is boolean and implements a non-linear function for the probability of treament
<br>
&nbsp;
\begin{align*}
\E(D\vert X=x)=\text{Pr}(D\vert X=x)&=0.1+0.89\cdot\sin(\pi x)\\
&\in \left[0,1\right]
\end{align*}
<br>
&nbsp;
Finally, the function returns a data frame with the variables $Y(0)$, $Y(1)$, $X$, $D$, and $X\cdot D$.

## Constant treatment effects (no heterogeneity) + linearity
For the first simulation we'll study a straightforward setup to illustrate where CATE and ATE coincide. We'll use the following specification.

* $f:\mathbb{R} \longrightarrow \mathbb{R},\quad x \mapsto x$ and $f=g=c=h$
<br>
&ensp;
* $\phi_1 = \gamma_1$ and $\phi_2 = \gamma_2 = 0$
<br>
&ensp;
* $\phi_0=5$, $\gamma_0 = 1$
<br>
&ensp;

Thus we obtain 
<br>
&ensp;
\begin{align*}
\delta&=\phi_0 - \gamma_0+\E\left[(\phi_1-\gamma_1)\cdot X \right]\\
&=\phi_0 - \gamma_0\\
&=4\\[10pt]
\delta(x)&=\E(\phi_0 - \gamma_0\vert X=x)\\
&=\phi_0 - \gamma_0\\
\end{align*}
<br>
&ensp;

In [4]:
set.seed(12553)
test <- sim(100)
head(test,10)

Y0,Y1,X,D,Y_obs,IntXD
<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
2.5457689,6.545769,0.38262576,0,2.5457689,0.00000000
0.9387381,4.938738,0.86127311,0,0.9387381,0.00000000
3.5509334,7.550933,0.86729329,1,7.5509334,0.86729329
0.0921096,4.092110,0.01338264,1,4.0921096,0.01338264
2.7765919,6.776592,0.60000625,0,2.7765919,0.00000000
2.4901825,6.490182,0.21668722,0,2.4901825,0.00000000
2.1165752,6.116575,0.56976284,1,6.1165752,0.56976284
0.1969798,4.196980,0.15011419,0,0.1969798,0.00000000
2.4790400,6.479040,0.82968150,0,2.4790400,0.00000000


This is to test whether git infrastructure works

In [11]:
x <- c("create_output_table.R", "create_output_table_JASA.R", "helper_functions\\sim.R", "helper_functions\\naive_ate.R", "helper_functions\\psw_ate_boot.R", "helper_functions\\ols_ate.R",
       "helper_functions\\knn_ate.R", "helper_functions\\rf_ate.R", "helper_functions\\MSE.R", "helper_functions\\knn_cate.R",
       "helper_functions\\ols_cate.R", "helper_functions\\rf_cate.R", "helper_functions\\rf_cate_JASA.R",
       "helper_functions\\ols_cate_JASA.R", "helper_functions\\simAW.R")
invisible(lapply(x, source))

In [7]:
create_output_table(n = 100, N = 100, B = 1000, k = 1, kC = 10, num_trees = 100,
                           het_linear = FALSE, random_assignment = TRUE,
                           non_linearY = FALSE, non_linearD = FALSE,
                           knn_se5 = TRUE, ols_se5 = TRUE, C_ols_se5 = TRUE,
                           C_knn_se5 = TRUE, boot5 = FALSE,
                           para = FALSE)

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


$ATE
       Description      Naive        OLS        PSW       k_NN         RF
1     avg_estimate     3.9990     4.0088     4.0237     4.0319     4.0142
2     sd_estimates     0.2526     0.4111     0.1875     0.2277     0.1901
3 avg_estimated_se     0.2700     0.4121         NA     0.2352     0.2225

$CATE
  Description        OLS       k_NN         RF
1     avg_mse    0.07674     0.2080    0.06422
2 avg_mean_se    0.19690     0.3514    0.29675

In [13]:
create_output_table_JASA(n = 100, N = 100, d = 10, num_trees = 200, kC = 7,
                       para = FALSE, C_knn_se5 = FALSE )

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


Description,OLS,k_NN,RF
<I<chr>>,<I<chr>>,<I<chr>>,<I<chr>>
avg_mse,1.52,0.709,1.12
